In [1]:
### Calculate new infer
from vdsm_util import get_blob_client, get_blobnames
from io import BytesIO, StringIO
from pyproj import Transformer
import pickle as pkl
import numpy as np

import time
import datetime
from dateutil.relativedelta import relativedelta
from azure.storage.blob import BlobServiceClient
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from azureml.core.run import Run
import scipy
import xarray as xr

import random
from sklearn.metrics import r2_score, accuracy_score
from azureml.core import Workspace
from azureml.core.authentication import MsiAuthentication
msi_auth = MsiAuthentication()

ws = Workspace(subscription_id="7704663e-8944-4300-b951-6462d41ab638", # put in .json
resource_group="arg-rt-pacedev-SpaceTech", # also in .json
workspace_name="aml-WS-spacetech001", # also
auth=msi_auth
)

keyvault = ws.get_default_keyvault()
conn_str = keyvault.get_secret("asartpacedevstvdsm001-connstr")

In [2]:
# import xarray as xr


In [3]:
# ## online run
# # run = Run.get_context()
# # conn_str = run.get_secret("asartpacedevstvdsm001-connstr")
# random_select = 20000
# veg_type = 'pv'
# f_year = '2021'
# f_prev_month = '01'
# month_model = 3
# last_col_name = f'pred_{month_model}mon'
# f_truth_month = 4  # to delta
# f_truth_year = 2021 # to delta
# infer_csvfolder = f'scoringoutput_csv/{f_year}/{f_prev_month}/pred{month_model}_{veg_type}.csv'



In [4]:
# infer_csvfolder

In [5]:
# blob_client = get_blob_client(filename=infer_csvfolder, conn_str=conn_str)
# df_infer = pd.read_csv(StringIO(blob_client.download_blob().content_as_text()))
# blob_client = BlobServiceClient.from_connection_string(conn_str)
# container_client = blob_client.get_container_client("data")

# # blob_names = list(container_client.list_blobs(name_starts_with=f'{f_truth_year}/{int(f_truth_month):02d}/fused/{veg_type}_v1/'))
# blob_names = container_client.list_blobs(name_starts_with=f'{f_truth_year}/{int(f_truth_month):02d}/fused/{veg_type}_v1/')

# truth_df = pd.DataFrame()
# for blob_name in blob_names:  ## to do random selection instead
#     blob_client = get_blob_client(filename=blob_name, conn_str=conn_str)
#     df = pd.read_csv(StringIO(blob_client.download_blob().content_as_text()))
#     truth_df = truth_df.append(df)

# random_df_truth = truth_df.sample(n=random_select, replace=False)
# random_df_truth_compare = random_df_truth[['filename', 'lat', 'lon', 'evi']]
# df_infer_compare = df_infer[['chainage_name', 'latitude', 'longitude', last_col_name]]


In [6]:
# df_infer_compare = df_infer_compare.rename(columns={'chainage_name':'filename', 'latitude':'lat', 'longitude':'lon'})
# df_infer_compare.head()


In [7]:
# # df_join_inner = random_df_truth_compare.merge(df_infer_compare, suffixes=("_truth", "_infer"), how='outer', on=['filename','lat', 'lon'], indicator=True)
# df_join_inner = random_df_truth_compare.merge(df_infer_compare, suffixes=("_truth", "_infer"), how='outer', right_on=['chainage_name','latitude', 'longitude'],
#                                                     left_on=['filename','lat', 'lon'], indicator=True)

In [62]:
df_join_inner.head()

filename        lat         lon       evi    chainage_name  \
0   TP_138_140.csv -21.655816  117.335779  0.051133   TP_138_140.csv   
1     DD_83_87.csv -21.264855  116.983474  0.534246     DD_83_87.csv   
2  LHR_481_485.csv -23.110547  119.366447  0.218942  LHR_481_485.csv   
3   PA_323_327.csv -22.944340  117.857781  0.328395   PA_323_327.csv   
4   TP_206_210.csv -22.099281  117.706560  0.771350   TP_206_210.csv   

    latitude   longitude  pred_3mon _merge  
0 -21.655816  117.335779   0.166711   both  
1 -21.264855  116.983474   0.487200   both  
2 -23.110547  119.366447   0.141265   both  
3 -22.944340  117.857781   0.390618   both  
4 -22.099281  117.706560   0.835893   both

In [8]:
# random_select = 16000
# veg_type = 'pv'
# f_year = '2021'
# f_prev_month = '01'

# fig, ax = plt.subplots(3,3, figsize=(10,10))

# i=0
# # for month_model in [3, 6]:
# for month_model in [3]:
#     for veg_type in ['pv']:

#     # for veg_type in ['pv', 'npv', 'bs']:
#         last_col_name = f'pred_{month_model}mon'
#         if month_model == 3:
#             f_truth_month = 4  # to delta
#             f_truth_year = 2021 # to delta
#         if month_model == 6:
#             f_truth_month = 7  # to delta
#             f_truth_year = 2021
#         if month_model == 12:
#             f_truth_month = 1  # to delta
#             f_truth_year = 2022
#         infer_csvfolder = f'scoringoutput_csv/{f_year}/{f_prev_month}/pred{month_model}_{veg_type}.csv'

#         blob_client = get_blob_client(filename=infer_csvfolder, conn_str=conn_str)
#         df_infer = pd.read_csv(StringIO(blob_client.download_blob().content_as_text()))
#         df_infer.info()
        
#         blob_client = BlobServiceClient.from_connection_string(conn_str)
#         container_client = blob_client.get_container_client("data")
#         blob_names = container_client.list_blobs(name_starts_with=f'{f_truth_year}/{int(f_truth_month):02d}/fused/{veg_type}_v1/')
#         # blob_names = container_client.list_blobs(name_starts_with=f'{f_truth_year}/{int(f_truth_month):02d}/staging/')

#         truth_df = pd.DataFrame()
#         for blob_name in blob_names:  ## to do random selection instead
#             blob_client = get_blob_client(filename=blob_name, conn_str=conn_str)
#             df = pd.read_csv(StringIO(blob_client.download_blob().content_as_text()))
#             truth_df = truth_df.append(df)

In [9]:
# truth_df.info()

In [10]:
# random_df_truth = truth_df   #.sample(n=random_select, replace=False)
# random_df_truth_compare = random_df_truth.loc[:, ['filename', 'lat', 'lon', 'evi']]
# df_infer_compare = df_infer.loc[:, ['chainage_name', 'latitude', 'longitude', last_col_name]]

# df_infer_compare.rename(columns={'chainage_name':'filename', 'latitude':'lat', 'longitude':'lon'}, inplace=True)
# df_infer_compare['lat'] = df_infer_compare['lat'].astype('object')
# df_infer_compare['lon'] = df_infer_compare['lon'].astype('object')


In [11]:
# random_df_truth_compare['lat'] = random_df_truth_compare['lat'].astype('object')
# random_df_truth_compare['lon'] = random_df_truth_compare['lon'].astype('object')
# random_df_truth_compare.info()

In [12]:
# df_join_inner = pd.merge(left=random_df_truth_compare, 
#                             right=df_infer_compare, how='inner', 
#                                             left_on=['filename','lat', 'lon'], 
#                                             right_on = ['filename','lat', 'lon'], 
#                                             copy=True, indicator=True,
#                                             suffixes=('_truth', '_infer'))




In [13]:
# df_join_inner.head()

In [14]:
# df_join_inner_clean = df_join_inner.dropna()
# df_join_inner_clean.info()

In [15]:
# print('---------')
# ### Bin EVI values into 4 bands to mimic PCI
# # bins = [0, 0.2, 0.225, 0.25, 0.275, 0.3, 0.325, 0.35, 0.375, 0.4, 0.425, 0.45, 0.475, 0.5, 0.55, 0.6]
# bins = [0, 0.2, 0.4, 0.6, 0.8]  

# names2 = list(range(len(bins)-1))
# df_join_inner_clean['pred_band'] = pd.cut(df_join_inner_clean[last_col_name], bins, labels=names2)
# df_join_inner_clean['actual_band'] = pd.cut(df_join_inner_clean['evi'], bins, labels=names2)


In [16]:
# df_join_inner_clean.head()

# y_pred = df_join_inner_clean['pred_band'].astype('int')
# y_true = df_join_inner_clean['actual_band'].astype('int')

In [17]:

# # cond = y_pred['pred_band'] > 0
# # acc =  accuracy_score(y_true[cond], y_pred[cond])
# cond = (y_pred > 0) & (y_true > 0)
# acc =  accuracy_score(y_true[cond], y_pred[cond])
# print("Acc: " + veg_type + " " + str(month_model) + " months: " + f'{acc:.2f}')

# r2 = r2_score(df_join_inner_clean['evi'], df_join_inner_clean[last_col_name])
# print("R2: "  + veg_type + " " + str(month_model) + " months: " +  f'{r2:.2f}')

In [2]:
random_select = 16000
veg_type = 'pv'
f_year = '2021'
f_prev_month = '01'

fig, ax = plt.subplots(3,3, figsize=(10,10))

i=0
for month_model in [3, 6]:
# for month_model in [3]:
    # for veg_type in ['pv']:

    for veg_type in ['pv', 'npv', 'bs']:
        last_col_name = f'pred_{month_model}mon'
        if month_model == 3:
            f_truth_month = 4  # to delta
            f_truth_year = 2021 # to delta
        if month_model == 6:
            f_truth_month = 7  # to delta
            f_truth_year = 2021
        if month_model == 12:
            f_truth_month = 1  # to delta
            f_truth_year = 2022
        infer_csvfolder = f'scoringoutput_csv/{f_year}/{f_prev_month}/pred{month_model}_{veg_type}.csv'

        blob_client = get_blob_client(filename=infer_csvfolder, conn_str=conn_str)
        df_infer = pd.read_csv(StringIO(blob_client.download_blob().content_as_text()))

        blob_client = BlobServiceClient.from_connection_string(conn_str)
        container_client = blob_client.get_container_client("data")
        blob_names = container_client.list_blobs(name_starts_with=f'{f_truth_year}/{int(f_truth_month):02d}/fused/{veg_type}_v1/')
        # blob_names = container_client.list_blobs(name_starts_with=f'{f_truth_year}/{int(f_truth_month):02d}/staging/')

        truth_df = pd.DataFrame()
        for blob_name in blob_names:  ## to do random selection instead
            blob_client = get_blob_client(filename=blob_name, conn_str=conn_str)
            df = pd.read_csv(StringIO(blob_client.download_blob().content_as_text()))
            truth_df = truth_df.append(df)
            # unpickled_ds = pkl.loads(blob_client.download_blob().content_as_bytes())
            # unpickled_ds['evi'] = 2.5*((unpickled_ds.nbart_nir_1 - unpickled_ds.nbart_red) / (unpickled_ds.nbart_nir_1 + 6*unpickled_ds.nbart_red - 7.5*unpickled_ds.nbart_blue +1))
            # da = xr.DataArray(coords={'y' : unpickled_ds['y'], 'x' : unpickled_ds['x']}, data=np.nanmean(unpickled_ds['evi'].values.clip(-1.5, 1.5), axis=0))
            # da_df = da.to_dataframe(name='evi').dropna()
            # da_df.reset_index(inplace=True)
            # da_df.drop_duplicates(subset=['y','x'], keep='first', inplace=True)
            # transformer = Transformer.from_crs("EPSG:3577", "EPSG:4326")
            # y_4326, x_4326 = transformer.transform(da_df.x.values, da_df.y.values)
            # da_df['lat'] = y_4326
            # da_df['lon'] = x_4326
            # da_df['filename'] = blob_name['name'].split('/')[3][:-7] + '.csv'
            # da_df.drop(columns = ['y', 'x'], inplace=True)
            # truth_df = truth_df.append(da_df)

        # print(truth_df)

        # truth_df = truth_df[truth_df['filename'].str.startswith('YA')]

        # random_df_truth = truth_df.sample(n=random_select, replace=False)
        # random_df_truth_compare = random_df_truth.loc[:, ['filename', 'lat', 'lon', 'evi']]
        # df_infer_compare = df_infer.loc[:, ['chainage_name', 'latitude', 'longitude', last_col_name]]

        # df_infer_compare = df_infer_compare.rename(columns={'chainage_name':'filename', 'latitude':'lat', 'longitude':'lon'})
        # df_join_inner = random_df_truth_compare.merge(df_infer_compare, how='right', on=['filename','lat', 'lon'], indicator=True)
        # # print(df_join_inner)

        random_df_truth = truth_df   #.sample(n=random_select, replace=False)
        random_df_truth_compare = random_df_truth.loc[:, ['filename', 'lat', 'lon', 'evi']]
        df_infer_compare = df_infer.loc[:, ['chainage_name', 'latitude', 'longitude', last_col_name]]

        df_infer_compare.rename(columns={'chainage_name':'filename', 'latitude':'lat', 'longitude':'lon'}, inplace=True)
        df_infer_compare['lat'] = df_infer_compare['lat'].astype('object')
        df_infer_compare['lon'] = df_infer_compare['lon'].astype('object')

        random_df_truth_compare['lat'] = random_df_truth_compare['lat'].astype('object')
        random_df_truth_compare['lon'] = random_df_truth_compare['lon'].astype('object')
        
        df_join_inner = pd.merge(left=random_df_truth_compare, 
                                    right=df_infer_compare, how='inner', 
                                                    left_on=['filename','lat', 'lon'], 
                                                    right_on = ['filename','lat', 'lon'], 
                                                    copy=True, indicator=True,
                                                    suffixes=('_truth', '_infer'))


        df_join_inner_clean = df_join_inner.dropna()
        # print(df_join_inner_clean)

        print('---------')
        ### Bin EVI values into 4 bands to mimic PCI
        # bins = [0, 0.2, 0.225, 0.25, 0.275, 0.3, 0.325, 0.35, 0.375, 0.4, 0.425, 0.45, 0.475, 0.5, 0.55, 0.6]
        bins = [0, 0.2, 0.4, 0.6, 0.8]  

        names2 = list(range(len(bins)-1))
        df_join_inner_clean['pred_band'] = pd.cut(df_join_inner_clean[last_col_name], bins, labels=names2)
        df_join_inner_clean['actual_band'] = pd.cut(df_join_inner_clean['evi'], bins, labels=names2)

        y_pred = df_join_inner_clean['pred_band'].astype('int')
        y_true = df_join_inner_clean['actual_band'].astype('int')
        # cond = y_pred['pred_band'] > 0
        # acc =  accuracy_score(y_true[cond], y_pred[cond])
        cond = (y_pred > 0) & (y_true > 0)
        acc =  accuracy_score(y_true[cond], y_pred[cond])
        print("Acc: " + veg_type + " " + str(month_model) + " months: " + f'{acc:.2f}')

        r2 = r2_score(df_join_inner_clean['evi'], df_join_inner_clean[last_col_name])
        print("R2: "  + veg_type + " " + str(month_model) + " months: " +  f'{r2:.2f}')

        # ax[i//3, i%3].scatter(df_join_inner['evi'], df_join_inner[f'pred_{month_model}mon'], s=2)
        sns.kdeplot(x='evi', y=last_col_name, data=df_join_inner_clean,
                     cmap="Blues", shade=True,
                     ax=ax[i//3, i%3])

        ax[i//3, i%3].plot([0, 1.], [0, 1.], c='black')
        ax[i//3, i%3].set_xlabel("Actuals")
        ax[i//3, i%3].set_ylabel("Predictions")
        ax[i//3, i%3].set_xlim(0, 1.)
        ax[i//3, i%3].set_ylim(0, 1.)
        ax[i//3, i%3].set_title(veg_type + " " + str(month_model) + " months")
        i+=1

plt.tight_layout()
plt.show()

---------
Acc: pv 3 months: 0.57
R2: pv 3 months: 0.43
---------
Acc: npv 3 months: 0.72
R2: npv 3 months: 0.41
---------
Acc: bs 3 months: 0.77
R2: bs 3 months: 0.47


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [5]:
# pv 3 months: 0.5531839893810333
# npv 3 months: 0.5621766092252563
# bs 3 months: 0.6267140115041288
# pv 6 months: 0.4662621270922428
# npv 6 months: 0.355464558050147
# bs 6 months: 0.3710929018146665
!ls

2021-01.pkl			train_fusion.py
__pycache__			train_fusion_pipeline.ipynb
azureml_environment.json	train_merge.py
conda_dependencies.yml		train_merge_pipeline.ipynb
data_drift_pipeline.ipynb	trigger_fusion_pipeline.py
data_drift_target_retrieval.py	trigger_infer_pipeline.py
era-mon-2015-2021.nc		trigger_pipeline.ipynb
infer_fusion.py			trigger_pipeline.py
infer_fusion_pipeline.ipynb	trigger_training.py
infer_merge.py			vdsm_inferdata_fusion_single.py
infer_merge_pipeline.ipynb	vdsm_new.yml
nc_test.pkl			vdsm_traindata_fusion.py
newtime_cal.ipynb		vdsm_traindata_fusion_parallel-copy.py
newtime_cal.ipynb.amltmp	vdsm_traindata_fusion_parallel.py
newtime_calv2.ipynb		vdsm_traindata_fusion_single.py
newtime_calv2.ipynb.amltmp	vdsm_util.py
pred3_pv.csv			vdsm_util.py.amltmp
register_training_data.py


In [3]:
filenc = 'era-mon-2015-2021.nc'
ds_env_all = xr.open_dataset(filenc)


In [4]:
with open('nc_test.pkl','wb') as f:
    pkl.dump(ds_env_all, f)


In [37]:
### help resolve the weird pkl uploding
filep ='nc_test.pkl'
# filep ='2021-01.pkl'
with open(filep, 'rb') as f:
    ds_env = pkl.load(f)
    # ds_env = pkl.loads(filep)

In [35]:
help(scipy.io.netcdf)

Help on module scipy.io.netcdf in scipy.io:

NAME
    scipy.io.netcdf - NetCDF reader/writer module.

DESCRIPTION
    This module is used to read and create NetCDF files. NetCDF files are
    accessed through the `netcdf_file` object. Data written to and from NetCDF
    files are contained in `netcdf_variable` objects. Attributes are given
    as member variables of the `netcdf_file` and `netcdf_variable` objects.
    
    This module implements the Scientific.IO.NetCDF API to read and create
    NetCDF files. The same API is also used in the PyNIO and pynetcdf
    modules, allowing these modules to be used interchangeably when working
    with NetCDF files.
    
    Only NetCDF3 is supported here; for NetCDF4 see
    `netCDF4-python <http://unidata.github.io/netcdf4-python/>`__,
    which has a similar API.

CLASSES
    builtins.object
        netcdf_file
        netcdf_variable
    
    class netcdf_file(builtins.object)
     |  A file object for NetCDF data.
     |  
     |  A `netc